# Actuators
This notebook introduces ...

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need Pinocchio, Gepetto-Viewer, SciPy for the solvers

In [6]:
import math
import time
import numpy as np
from numpy.linalg import norm,inv,pinv,svd,eig
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs
from tp4.meshcat_viewer_wrapper import *

In [7]:
robot = robex.load('solo12')
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [8]:
viz.display(robot.q0)


Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [15]:
# %load -r 28- tp2/floating.py

robot.feetIndexes = [ robot.model.getFrameId(frameName) for frameName in ['HR_FOOT','HL_FOOT','FR_FOOT','FL_FOOT' ] ]

# --- Add box to represent target
colors = ['red','blue','green','magenta']
for color in colors:
    viz.addSphere("world/%s"%color, .05, color)
    viz.addSphere("world/%s_des"%color, .05, color)

#
# OPTIM 6D #########################################################
#

targets = [
    np.array(  [-0.7, -0.2,  1.2]),
    np.array(  [-0.3,  0.5,  0.8]),
    np.array(  [ 0.3,  0.1, -0.1]),
    np.array(  [ 0.9,  0.9,  0.5])
    ]
for i in range(4): targets[i][2]+=1

def cost(q):
    '''Compute score from a configuration'''
    cost = 0.
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        cost += norm(p_i-targets[i])**2
    cost += 100*(1-norm(q[3:7]))**2
    return cost

def callback(q):
    viz.applyConfiguration('world/box', Mtarget)

    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        viz.applyConfiguration('world/%s'%colors[i], p_i)
        viz.applyConfiguration('world/%s_des'%colors[i], list(targets[i])+[1,0,0,0])

    viz.display(q)
    time.sleep(1e-1)

Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)



Optimization terminated successfully.
         Current function value: 0.614958
         Iterations: 74
         Function evaluations: 1620
         Gradient evaluations: 81


In [13]:
norm(qopt[3:7])

3.4645610585829423

## Configuration of parallel robots
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [18]:
import tp2.load_ur5_parallel as robex2
robot = robex2.load()

In [19]:
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [20]:
viz.display(robot.q0)

In [21]:
[w, h, d] = [0.5, 0.5, 0.005]
color = [red, green, blue, transparency] = [1, 1, 0.78, .8]
viz.addBox('world/robot0/toolplate', [w, h, d], color)
Mtool = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool)

The 4 legs of the robot are loaded in a single robot model. The 4 effector placements are computed by:

In [22]:
effIdxs = [ robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]
robot.framePlacement(robot.q0,effIdxs[0])

  R =
   -0.707107    -0.707107 -3.46245e-12
    0.707107    -0.707107 -3.46236e-12
-6.12323e-17 -4.89664e-12            1
  p = 0.306122 0.160483 0.749342

The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in with the configuration *robot.q0* and the plate placement *Mtool*. To be valid, a configuration *q* must satisfy these 4 relative placement constraints.

Consider now that the orientation of the tool plate is given by the following quaternion, with the translation that you like (see [the notebook about rotations if you need more details](appendix1_quaternions.ipynb)): 



In [23]:
quat = pin.Quaternion(0.7,0.2,0.2,0.6).normalized()
print(quat.matrix())

[[ 0.13978495 -0.8172043   0.55913978]
 [ 0.98924731  0.13978495 -0.04301075]
 [-0.04301075  0.55913978  0.82795699]]


**Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met** for the new orientation of the plate.